In [ ]:
import requests
import pandas as pd
from io import StringIO  # Needed to wrap HTML text for pandas.read_html()
from minio import Minio
from io import BytesIO
import os
from dotenv import load_dotenv

In [46]:
url = "https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2024_-_S%C3%A9rie_A"

In [47]:
# 1. Configurações (via .env)
load_dotenv()
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")
MINIO_ACCESS = os.getenv("MINIO_ACCESS_KEY")
MINIO_SECRET = os.getenv("MINIO_SECRET_KEY")
BUCKET_NAME = os.getenv("MINIO_BUCKET")

In [48]:
# Fetch HTML content
response = requests.get(url)
response.raise_for_status()  # Check for request errors
html = response.text

In [49]:
# Parse tables (with StringIO wrapper)
tables = pd.read_html(StringIO(html))

In [35]:
# Display information about found tables
print(f"Found {len(tables)} tables on the page")

Found 28 tables on the page


In [36]:
display(tables)

[                           Brasileirão 2024 - Série A  \
 0   Campeonato Brasileiro de Futebol de 2024 - Sér...   
 1                                               Dados   
 2                                       Participantes   
 3                                         Organização   
 4                                             Período   
 5                                             Gol(o)s   
 6                                            Partidas   
 7                                               Média   
 8                                             Campeão   
 9                                        Vice-campeão   
 10                                       3.º colocado   
 11                                       4.º colocado   
 12                                       Rebaixado(s)   
 13                                    Melhor marcador   
 14                                      Melhor ataque   
 15                                      Melhor defesa   
 16           

In [37]:
# Example: Show the head of a table
if len(tables) > 0:
    display(tables[5].head())

,Pos,Equipevde,Pts,J,V,E,D,GP,GC,SG,Classificação ou descenso
0,1,Botafogo (C),79,38,23,10,5,59,29,+30,Fase de grupos da Copa Libertadores de 2025[a]
1,2,Palmeiras,73,38,22,7,9,60,33,+27,Fase de grupos da Copa Libertadores de 2025[a]
2,3,Flamengo,70,38,20,10,8,61,42,+19,Fase de grupos da Copa Libertadores de 2025[a]
3,4,Fortaleza,68,38,19,11,8,53,39,+14,Fase de grupos da Copa Libertadores de 2025[a]
4,5,Internacional,65,38,18,11,9,53,36,+17,Fase de grupos da Copa Libertadores de 2025[a]


In [50]:
df = tables[5]
display(df)

,Pos,Equipevde,Pts,J,V,E,D,GP,GC,SG,Classificação ou descenso
0,1,Botafogo (C),79,38,23,10,5,59,29,+30,Fase de grupos da Copa Libertadores de 2025[a]
1,2,Palmeiras,73,38,22,7,9,60,33,+27,Fase de grupos da Copa Libertadores de 2025[a]
2,3,Flamengo,70,38,20,10,8,61,42,+19,Fase de grupos da Copa Libertadores de 2025[a]
3,4,Fortaleza,68,38,19,11,8,53,39,+14,Fase de grupos da Copa Libertadores de 2025[a]
4,5,Internacional,65,38,18,11,9,53,36,+17,Fase de grupos da Copa Libertadores de 2025[a]
5,6,São Paulo,59,38,17,8,13,53,43,+10,Fase de grupos da Copa Libertadores de 2025[a]
6,7,Corinthians,56,38,15,11,12,54,45,+9,Segunda fase da Copa Libertadores de 2025
7,8,Bahia,53,38,15,8,15,49,49,0,Segunda fase da Copa Libertadores de 2025
8,9,Cruzeiro,52,38,14,10,14,43,41,+2,Fase de grupos da Copa Sul-Americana de 2025
9,10,Vasco da Gama,50,38,14,8,16,43,56,−13,Fase de grupos da Copa Sul-Americana de 2025


In [56]:
# Connect to Minio
minio_client = Minio(
    MINIO_ENDPOINT,
    access_key= MINIO_ACCESS,
    secret_key= MINIO_SECRET,
    secure= False                   # True se estiver usando HTTPS
    
)


In [57]:

# Minimal connection test
try:
    minio_client.list_buckets()  # Simplest API call to test connectivity
    print("MinIO connection working")
except Exception as e:
    print(f"Connection failed: {str(e)}")

MinIO connection working


In [58]:
# create bucket if not exist
if not minio_client.bucket_exists(BUCKET_NAME):
    minio_client.make_bucket(BUCKET_NAME)

In [60]:
# Prepare and upload Parquet file
parquet_buffer = BytesIO()
df.to_parquet(parquet_buffer, engine='pyarrow', compression='snappy')
parquet_buffer.seek(0)

0

In [61]:
minio_client.put_object(
    BUCKET_NAME,
    "campeonato_brasileiro_2024.parquet",
    parquet_buffer,
    length=parquet_buffer.getbuffer().nbytes,
    content_type="application/octet-stream"
)

In [62]:
print(f"Successfully saved as {BUCKET_NAME}/campeonato_brasileiro_2024.parquet")

Successfully saved as test-bucket/campeonato_brasileiro_2024.parquet
